In [ ]:
import os
import glob
import json
import torch
import subprocess
import numpy as np
import soundfile as sf
import whisper
import kss
from dataclasses import dataclass
from typing import List
from openai import OpenAI
from transformers import AutoTokenizer, AutoModel
from df.enhance import enhance, init_df, load_audio, save_audio

# =======================
# ⚙️ 설정값
# =======================
SOLAR_API_KEY = "up_EWqzIcLH0bVwkJgPRyb4CYrJCCEwy"
SOLAR_BASE_URL = "https://api.upstage.ai/v1/solar"
MODEL_ID = "solar-pro2"
VIDEO_DIR = "/home/stu/ai_project/시연"     
RESULT_DIR = "/home/stu/ai_project/시연"      
os.makedirs(RESULT_DIR, exist_ok=True)

@dataclass
class SentenceSegment:
    index: int
    start: float
    end: float
    text: str

# =======================
# 1. 오디오/비디오 처리 함수
# =======================
def extract_audio_from_video(video_path: str) -> str:
    audio_path = video_path.rsplit('.', 1)[0] + ".wav"
    if os.path.exists(audio_path):
        return audio_path
    command = ["ffmpeg", "-y", "-i", video_path, "-ac", "1", "-ar", "48000", "-vn", audio_path]
    subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
    return audio_path

def remove_noise_and_add_comfort(input_path: str) -> str:
    model, df_state, _ = init_df()
    audio, _ = load_audio(input_path, sr=df_state.sr())
    enhanced_audio = enhance(model, df_state, audio)
    
    clean_path = input_path.replace(".wav", "_clean.wav")
    save_audio(clean_path, enhanced_audio, df_state.sr())
    
    del model
    if torch.cuda.is_available(): torch.cuda.empty_cache()

    data, samplerate = sf.read(clean_path)
    noise_level = 0.001
    noise = np.random.randn(*data.shape)
    data_with_cn = data + (noise * noise_level)
    
    final_path = input_path.replace(".wav", "_final.wav")
    sf.write(final_path, data_with_cn, samplerate)
    
    if os.path.exists(clean_path): os.remove(clean_path)
    return final_path

def align_sentences_with_words(full_text: str, all_words: List[dict]) -> List[dict]:
    sentences = kss.split_sentences(full_text)
    aligned_segments = []
    word_idx = 0
    max_words = len(all_words)
    
    for sent in sentences:
        if word_idx >= max_words: break
        target_chars = sent.replace(" ", "")
        accumulated_chars = ""
        start_time = all_words[word_idx]['start']
        end_time = start_time
        
        while word_idx < max_words:
            word_info = all_words[word_idx]
            clean_word = word_info['word'].replace(" ", "")
            accumulated_chars += clean_word
            end_time = word_info['end']
            word_idx += 1
            if len(accumulated_chars) >= len(target_chars): break
        
        aligned_segments.append({"start": start_time, "end": end_time, "text": sent})
    return aligned_segments
    
def process_video_to_sentences(video_path: str) -> List[SentenceSegment]:
    raw_audio = extract_audio_from_video(video_path)
    voice_path = remove_noise_and_add_comfort(raw_audio)
    
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model = whisper.load_model("large-v3-turbo", device=device)
    result = model.transcribe(voice_path, language="ko", word_timestamps=True)
    
    all_words = []
    for segment in result['segments']:
        for word in segment['words']:
            all_words.append({"word": word['word'].strip(), "start": word['start'], "end": word['end']})
    
    aligned_data = align_sentences_with_words(result['text'], all_words)
    
    segments = []
    for idx, s in enumerate(aligned_data, start=1):
        segments.append(SentenceSegment(idx, s["start"], s["end"], s["text"]))
    
    del model
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    
    if os.path.exists(raw_audio): os.remove(raw_audio)
    if os.path.exists(voice_path): os.remove(voice_path)
    
    return segments

# =======================
# 2. Solar 분석 & 텐서 추출 함수 
# =======================
def analyze_with_solar(structured_data):
    print(f"   ☀️ Solar 분석 시작 ({len(structured_data)} 문장)...")
    client = OpenAI(api_key=SOLAR_API_KEY, base_url=SOLAR_BASE_URL)
    full_script = "\n".join([f"[{item['index']}] {item['text']}" for item in structured_data])
    
    final_analysis_list = []
    BATCH_SIZE = 20
    
    for i in range(0, len(structured_data), BATCH_SIZE):
        batch = structured_data[i : i + BATCH_SIZE]
        script_for_llm = "\n".join([f"[{item['index']}] {item['text']}" for item in batch])

        user_content = f"""
[전체 대본 (Context)]
{full_script}

[이번에 분석할 파트 (Target)]
{script_for_llm}
"""
        system_prompt = """
너는 한국어 발표·스피치 코칭 전문가이며, Whisper가 추출한 대본 전체를 분석해
각 문장이 청중을 설득하는 데 얼마나 ‘핵심적이고 강조 가치’를 갖는지 정량적으로 평가한다.

[매우 중요한 규칙]
1. 입력된 문장의 개수와 순서를 정확히 지켜라.
2. JSON의 analysis 리스트 길이는 입력 문장 개수와 정확히 같아야 한다.
3. analysis 리스트의 각 항목에는 반드시 index 필드를 포함해라.
4. start_sec, end_sec 같은 시간 정보는 출력하지 마라.
5. 오직 index, corrected, emphasis_score, key_word, reason 필드만 출력하라.

[Correction 규칙]
- 명백한 오타만 수정하라.

[Emphasis Score 규칙 (0.00 ~ 1.00)]
- 0.81~1.00: 핵심 메시지 (강조 필수)
- 0.61~0.80: 중요 예시/질문
- 0.00~0.60: 일반 정보

[출력 형식]
- 오직 하나의 JSON 객체만 출력한다. 예: {"analysis": [...]}
"""
        try:
            completion = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt}, 
                    {"role": "user", "content": user_content} 
                ],
                model=MODEL_ID, temperature=0.0, response_format={"type": "json_object"}
            )
            res = json.loads(completion.choices[0].message.content)
            final_analysis_list.extend(res.get("analysis", []))
            print(f"      -> {i+1} ~ {min(i+BATCH_SIZE, len(structured_data))} 문장 완료")
        except Exception as e:
            print(f"      ⚠️ Solar API Err: {e}")

    llm_map = {item.get("index"): item for item in final_analysis_list}
    final_result = []

    for origin in structured_data:
        idx = origin["index"]
        merged = {
            "index": idx,
            "start_sec": origin["start"],
            "end_sec": origin["end"]
        }
        if idx in llm_map:
            t = llm_map[idx]
            merged.update({
                "corrected": t.get("corrected", origin["text"]),
                "emphasis_score": float(t.get("emphasis_score", 0.0)),
                "key_word": t.get("key_word", ""),
                "reason": t.get("reason", "")
            })
        else:
            merged.update({
                "corrected": origin["text"],
                "emphasis_score": 0.0,
                "key_word": "",
                "reason": "API 누락"
            })
        final_result.append(merged)
        
    return {"analysis": final_result}

def extract_tensors(json_data):
    print(f"   💎 RoBERTa 텐서 추출 중...")
    try:
        tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")
        model = AutoModel.from_pretrained("klue/roberta-large", output_hidden_states=True)
    except:
        print("   ❌ RoBERTa 모델 로드 실패")
        return []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    tensor_results = []
    for item in json_data.get("analysis", []):
        text = item.get('corrected', '').strip()
        if text:
            inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128).to(device)
            with torch.no_grad():
                outputs = model(**inputs)
            last_4_layers = outputs.hidden_states[-4:]
            cls_list = [layer[:,0,:] for layer in last_4_layers]
            sum_tensor = torch.stack(cls_list).sum(dim=0).cpu()
            tensor_results.append(sum_tensor) 
        else:
            tensor_results.append(torch.zeros(1, 1024))
            
    return tensor_results

# =======================
# 3. 메인 실행 루프 
# =======================
if __name__ == "__main__":
    target_extensions = ['*.mp4', '*.avi', '*.mkv', '*.mov']
    video_files = []
    for ext in target_extensions:
        video_files.extend(glob.glob(os.path.join(VIDEO_DIR, ext)))
    video_files.sort()
    
    print(f"📂 총 {len(video_files)}개의 영상을 처리합니다.\n")

    total_json_list = []
    total_tensor_list = []

    for i, video_path in enumerate(video_files, 1):
        filename = os.path.basename(video_path) 
        file_id = filename.rsplit('.', 1)[0]    
        
        print(f"[{i}/{len(video_files)}] 처리 중: {filename}")
        
        try:
            sentences = process_video_to_sentences(video_path)
            structured_data = [{"index": s.index, "text": s.text, "start": s.start, "end": s.end} for s in sentences]
            
            final_json = analyze_with_solar(structured_data)
            
            tensors = extract_tensors(final_json)
            
            current_analysis_list = final_json.get("analysis", [])
            for item in current_analysis_list:
                item["video_path"] = video_path  
                item["video_filename"] = filename 
            
            total_json_list.extend(current_analysis_list)
            if tensors:
                total_tensor_list.extend(tensors)

            save_json_path = os.path.join(RESULT_DIR, f"{file_id}_text.json")
            save_tensor_path = os.path.join(RESULT_DIR, f"{file_id}_tensor.pt")
            
            with open(save_json_path, "w", encoding="utf-8") as f:
                json.dump(final_json, f, ensure_ascii=False, indent=4)
            if tensors:
                torch.save(tensors, save_tensor_path)
            
            print(f"   ✅ 개별 처리 완료: {len(current_analysis_list)} 문장\n")
            
        except Exception as e:
            print(f"   ❌ {filename} 오류 발생: {e}")
            import traceback
            traceback.print_exc()

    # =======================
    # 4. 최종 병합 파일 저장
    # =======================
    print("=" * 50)
    print("🚀 모든 영상 처리 완료! 병합 파일 생성 중...")

    final_merged_json = {"analysis": total_json_list}
    merged_json_path = os.path.join(RESULT_DIR, "total_text.json")
    with open(merged_json_path, "w", encoding="utf-8") as f:
        json.dump(final_merged_json, f, ensure_ascii=False, indent=4)
    
    merged_tensor_path = os.path.join(RESULT_DIR, "total_tensor.pt")
    if total_tensor_list:
        torch.save(total_tensor_list, merged_tensor_path)
        print(f"✅ [통합 완료] 텐서 개수: {len(total_tensor_list)}개")
        print(f"   💾 저장 경로: {merged_tensor_path}")
    else:
        print("⚠️ 저장할 텐서가 없습니다.")

    print(f"✅ [통합 완료] JSON 데이터 개수: {len(total_json_list)}개")
    print(f"   💾 저장 경로: {merged_json_path}")

/home/stu/anaconda3/envs/ai_project_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/stu/anaconda3/envs/ai_project_env/lib/python3.10/site-packages/df/io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData


📂 총 3개의 영상을 처리합니다.

[1/3] 처리 중: 20251215_193652.mp4
2025-12-15 20:17:11 | INFO     | DF | Running on torch 2.6.0+cu124
2025-12-15 20:17:11 | INFO     | DF | Running on host n9
2025-12-15 20:17:11 | INFO     | DF | Loading model settings of DeepFilterNet3
2025-12-15 20:17:11 | INFO     | DF | Using DeepFilterNet3 model at /home/stu/.cache/DeepFilterNet/DeepFilterNet3
2025-12-15 20:17:11 | INFO     | DF | Initializing model `deepfilternet3`


fatal: not a git repository (or any of the parent directories): .git


2025-12-15 20:17:11 | INFO     | DF | Found checkpoint /home/stu/.cache/DeepFilterNet/DeepFilterNet3/checkpoints/model_120.ckpt.best with epoch 120
2025-12-15 20:17:12 | INFO     | DF | Running on device cuda:0
2025-12-15 20:17:12 | INFO     | DF | Model loaded


[Kss]: Because there's no supported C++ morpheme analyzer, Kss will take pecab as a backend. :D
For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://github.com/hyunwoongko/python-mecab-kor
- konlpy.tag.Mecab: https://konlpy.org/en/latest/api/konlpy.tag/#mecab-class



...

   ✅ 개별 처리 완료: 23 문장

[2/3] 처리 중: 20251215_194052.mp4
2025-12-15 20:18:50 | INFO     | DF | Loading model settings of DeepFilterNet3
2025-12-15 20:18:50 | INFO     | DF | Using DeepFilterNet3 model at /home/stu/.cache/DeepFilterNet/DeepFilterNet3
2025-12-15 20:18:50 | INFO     | DF | Initializing model `deepfilternet3`
2025-12-15 20:18:50 | INFO     | DF | Found checkpoint /home/stu/.cache/DeepFilterNet/DeepFilterNet3/checkpoints/model_120.ckpt.best with epoch 120
2025-12-15 20:18:50 | INFO     | DF | Running on device cuda:0
2025-12-15 20:18:50 | INFO     | DF | Model loaded
   ☀️ Solar 분석 시작 (23 문장)...
      -> 1 ~ 20 문장 완료
      -> 21 ~ 23 문장 완료
   💎 RoBERTa 텐서 추출 중...


/home/stu/anaconda3/envs/ai_project_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


   ✅ 개별 처리 완료: 23 문장

[3/3] 처리 중: 20251215_195125.mp4
2025-12-15 20:20:40 | INFO     | DF | Loading model settings of DeepFilterNet3
2025-12-15 20:20:40 | INFO     | DF | Using DeepFilterNet3 model at /home/stu/.cache/DeepFilterNet/DeepFilterNet3
2025-12-15 20:20:40 | INFO     | DF | Initializing model `deepfilternet3`
2025-12-15 20:20:40 | INFO     | DF | Found checkpoint /home/stu/.cache/DeepFilterNet/DeepFilterNet3/checkpoints/model_120.ckpt.best with epoch 120
2025-12-15 20:20:40 | INFO     | DF | Running on device cuda:0
2025-12-15 20:20:40 | INFO     | DF | Model loaded
   ☀️ Solar 분석 시작 (25 문장)...
      -> 1 ~ 20 문장 완료
      -> 21 ~ 25 문장 완료
   💎 RoBERTa 텐서 추출 중...


/home/stu/anaconda3/envs/ai_project_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


   ✅ 개별 처리 완료: 25 문장

🚀 모든 영상 처리 완료! 병합 파일 생성 중...
✅ [통합 완료] 텐서 개수: 71개
   💾 저장 경로: /home/stu/ai_project/시연연/total_tensor.pt
✅ [통합 완료] JSON 데이터 개수: 71개
   💾 저장 경로: /home/stu/ai_project/시연연/total_text.json


In [ ]:
import os
import glob
import json
import torch
from tqdm import tqdm

# ===== 설정 =====
TEXT_OUTPUT_DIR = "/home/stu/ai_project/시연" 
SAVE_DIR = "/home/stu/ai_project/시연"

os.makedirs(SAVE_DIR, exist_ok=True)

def merge_text_data(input_dir, save_dir):
    json_files = sorted(glob.glob(os.path.join(input_dir, "*_text.json")))
    
    total_json_list = []
    total_tensor_list = []
    
    print(f"📂 '{input_dir}' 에서 파일 병합을 시작합니다...")
    print(f"   - 발견된 파일 수: {len(json_files)} 세트")

    for json_path in tqdm(json_files):
        filename = os.path.basename(json_path)
        file_id = filename.replace("_text.json", "")
        
        tensor_path = os.path.join(input_dir, f"{file_id}_tensor.pt")
        
        if not os.path.exists(tensor_path):
            print(f"   ⚠️ 경고: 텐서 파일을 찾을 수 없음 (Skip) - {file_id}")
            continue
            
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                analysis_list = data.get("analysis", [])
            
            tensors = torch.load(tensor_path)
            
            if len(analysis_list) != len(tensors):
                print(f"   ❌ 오류: 문장 개수와 텐서 개수 불일치 ({file_id})")

                continue

            for item in analysis_list:
                item["video_filename"] = file_id 

            total_json_list.extend(analysis_list)
            
            if isinstance(tensors, list):
                total_tensor_list.extend(tensors)
            else:
                for i in range(tensors.size(0)):
                    total_tensor_list.append(tensors[i])

        except Exception as e:
            print(f"   ❌ 처리 중 에러 발생 ({file_id}): {e}")

    # ===== 저장 =====
    print("\n💾 병합된 데이터 저장 중...")
    
    final_json_path = os.path.join(save_dir, "total_text.json")
    with open(final_json_path, "w", encoding="utf-8") as f:
        json.dump({"analysis": total_json_list}, f, ensure_ascii=False, indent=4)

    final_tensor_path = os.path.join(save_dir, "total_tensor.pt")
    torch.save(total_tensor_list, final_tensor_path)
    
    print(f"✅ 완료!")
    print(f"   - JSON: {final_json_path} (총 {len(total_json_list)} 문장)")
    print(f"   - Tensor: {final_tensor_path} (총 {len(total_tensor_list)} 개)")

if __name__ == "__main__":
    merge_text_data(TEXT_OUTPUT_DIR, SAVE_DIR)

📂 '/home/stu/ai_project/시연' 에서 파일 병합을 시작합니다...
   - 발견된 파일 수: 3 세트


100%|██████████| 3/3 [00:00<00:00, 70.61it/s]


💾 병합된 데이터 저장 중...
✅ 완료!
   - JSON: /home/stu/ai_project/시연/total_text.json (총 71 문장)
   - Tensor: /home/stu/ai_project/시연/total_tensor.pt (총 71 개)
